**YOUR NAMES HERE**

Spring 2020

CS 443: Computational Neuroscience

Project 4: Motion estimation

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt

plt.style.use(['seaborn-colorblind', 'seaborn-darkgrid'])
plt.rcParams.update({'font.size': 20})

np.set_printoptions(suppress=True, precision=3)

# Automatically reload external modules
%load_ext autoreload
%autoreload 2

## Task 4) Build the deep network layers (Layers 3-6)

### 4a. Layer 3: Short-range filter cells

#### (i) Implement helper methods

Layers 3+ do convolution to compute their `netIn`, so you will create a method that handles the construction of all the various kernels.

- Implement `make_kernels` in `MotionNet`. This makes all the excitatory/inhibitory convolutional kernels in Levels 3+. For now, just create the Level 3 excitatory anisotropic Gaussian kernels. The elongated direction should align with the preferred direction of each neuron. I suggest a filter size of (13, 13) or thereabouts (small relative to the spatial extent of the video). Remember to wrap code in level-specific boolean.
- Call `make_kernels` in constructor to build all the kernels whenever you create a network.

#### (ii) Implement and plot the layer output

Here are the steps we will perform over and over again to add a new layer.

- Implement `d_short_range_filter`.
- Update `update_net` to integrate cell activity in Level 3 (remember to put your code in a boolean if statement).
- Simulate the noise-free rightward motion RDK video (`test_rdk_0`) with network layers 1, 2, and 3.
- Run the direction decode test below.
- In the cell below, compute the rectifed layer output of the short-range filter (srf) cells.
- Plot the activation of the rectified output of all directions using `plot_act_image_grid`. Compare for consistency to `right_test_level3_grid.mov`. 
    - Note that I am plotting starting at time step 9 to the end in steps of 10.
- Plot the rectified srf cells using your vector sum plot. Compare for consistency with `right_test_level3_vectors.mov`

In [ ]:

# Set up input
np.random.seed(0)

# Define parameters

# Simulate net


# Compute layer 3 output activation


In [ ]:
# Your grid of image plots of the layer 3 cells output over time


#### Test: Decoded directional transient directions

**IMPORTANT NOTE:** Your exact values **WILL** differ and thats ok (due to parameter choices and different implementation decisions). You simply want to get rough qualitative agreement.

In [ ]:
print(f'Your decoded evidence for each direction at the end of the simulation is\n{net.decode_direction(net.srf_cells, -1)}\nand should roughly look like\n[112.433 106.568 102.521 102.601   0.5   105.73  105.342 107.313]')

In [ ]:
# Your vector sum plot of the layer 3 output over time


### 4b. Layer 4: Spatial and directional competition cells in MT

Adding layer 4 follows the same procedure as before to add a new layer:

- Update `make_kernels` to build Level 4 excitatory and inhibitory kernels. Kernel sizes should be between 1.5-2x those used in Layer 3, though remember we want to implement on-center/off-surround organization (excit filter ~same size as Layer 3, inhib filter a bit bigger with larger sigma).
- Implement `d_competition_layer`.
- Update `update_net` to integrate cell activity in Level 4 (remember to put your code in a boolean if statement).
- Simulate the noise-free rightward motion RDK video (`test_rdk_0`) with network layers 1, 2, 3, and 4.
- Run the direction decode test below.
- In the cell below, compute the rectifed competition cell output.
- Plot the activation of the rectified output of all directions using `plot_act_image_grid`. Compare for consistency to `right_test_level4_grid.mov`.
    - Note that I am plotting starting at time step 9 to the end in steps of 10. (*Please ignore the drifting of the placement of the second row of plots*).
- Plot the rectified competition cells using your vector sum plot. Compare for consistency with `right_test_level4_vectors.mov`

In [ ]:
# Define meta parameters

# Set up input
np.random.seed(0)

# Define parameters

# Simulate net


# Compute layer 4 output activation


In [ ]:
# Your grid of image plots of the layer 4 cells output over time


#### Test: Decoded directional transient directions

**IMPORTANT NOTE:** Your exact values **WILL** differ and thats ok (due to parameter choices and different implementation decisions). You simply want to get rough qualitative agreement.

In [ ]:
print(f'Your decoded evidence for each direction at the end of the simulation is\n{net.decode_direction(net.comp_cells, -1)}\nand should roughly look like\n[118.35   47.927  34.174  42.399   0.     48.013  38.499  46.949]')

In [ ]:
# Your vector sum plot of the layer 4 output over time


### 4e. Level 5: Long-range filter in MT

Adding layer 5 follows the same procedure as before to add a new layer:

- Update `make_kernels` to build Level 5 excitatory kernels. Filter size should be ~4x Layer 3 SRF (hence the name :)
- Implement `d_long_range`.
- Update `update_net` to integrate cell activity in Level 5 (remember to put your code in a boolean if statement).
- Simulate the noise-free rightward motion RDK video (`test_rdk_0`) with network layers 1, 2, 3, 4, and 5.
- Run the direction decode test below.
- In the cell below, rectify the output of the long-range filter cells.
- Plot the activation of the rectified output of all directions using `plot_act_image_grid`. Compare for consistency to `right_test_level5_grid.mov`.
    - Note that I am plotting starting at time step 9 to the end in steps of 10. (*Please ignore the drifting of the placement of the second row of plots*).
- Plot the rectified long-range filter cells using your vector sum plot. Compare for consistency with `right_test_level5_vectors.mov`

In [ ]:
# Define meta parameters

# Set up input
np.random.seed(0)

# Define parameters

# Simulate net

# Compute layer 5 output activation


In [ ]:
# Your grid of image plots of the layer 5 cells output over time


#### Test: Decoded directional transient directions

**IMPORTANT NOTE:** Your exact values **WILL** differ and thats ok (due to parameter choices and different implementation decisions). You simply want to get rough qualitative agreement.

In [ ]:
print(f'Your decoded evidence for each direction at the end of the simulation is\n{net.decode_direction(net.lr_cells, -1)}\nand should roughly look like\n[212.698  91.528  67.421  80.858   0.     90.174  73.545  89.044]')

In [ ]:
# Your vector sum plot of the layer 5 output over time


### 4f. Level 6: Direction grouping in MSTd

Adding layer 6 follows the same procedure as before to add a new layer, **with the exception of implementing MSTd-MT feedback (Layer 6-> 5)**

#### (i) Implement and test MSTd-MT feedback (Layer 6 -> Layer 5)

- `make_mstd_fb_wts`. Weights for directional inhibition in MSTd and feedback from MSTd to MT (long-range filters). Each cell recieves no inhibition from other neurons with the same direction preference (0 wt). Each cell recieves the most inhibition from the opponent direction (2 wt). All other directions recieve equally weighted moderate inhibition (1 wt). **Run the test code below**.
- Create your MSTd feedback weights in the constructor.
- Implement `mstd_fb` **then run the test code below**.

#### (ii) Test `make_mstd_fb_wts`

In [ ]:
dt = 0.1
n_dirs = 8

test_net = MotionNet(dt,
                     n_dirs,
                     lvl1_params=None,
                     lv1_hgate_params=None,
                     do_lvl2=False,
                     do_lvl3=False,
                     do_lvl4=False,
                     do_lvl5=False,
                     do_lvl6=False)
wts = test_net.make_mstd_fb_wts()
print('Your MSTd inhibitory weight matrix is:\n', wts)

It should be:

    [[0. 1. 1. 1. 2. 1. 1. 1.]
     [1. 0. 1. 1. 1. 2. 1. 1.]
     [1. 1. 0. 1. 1. 1. 2. 1.]
     [1. 1. 1. 0. 1. 1. 1. 2.]
     [2. 1. 1. 1. 0. 1. 1. 1.]
     [1. 2. 1. 1. 1. 0. 1. 1.]
     [1. 1. 2. 1. 1. 1. 0. 1.]
     [1. 1. 1. 2. 1. 1. 1. 0.]]

#### (iii) Test `mstd_fb`

In [ ]:
dt = 0.1
n_dirs = 8
n_frames = 1

net = MotionNet(dt=dt,
                n_dirs=n_dirs,
                lvl1_params=None,
                lv1_hgate_params=None,
                lvl2_inter_params=None,
                lvl2_params=None,
                lvl3_params=None,
                lvl3_excit_ker_params=None,
                lvl4_params=None,
                lvl4_excit_ker_params=None,
                lvl4_inhib_ker_params=None,
                lvl5_params=None,
                lvl5_excit_ker_params=None,
                lvl6_params=None,
                lvl6_inhib_ker_params=None,
                do_lvl2=False,
                do_lvl3=False,
                do_lvl4=False,
                do_lvl5=False,
                do_lvl6=False
                )
net.mstd_inhib_ker = np.array([[0, 1, 0], [1, 2, 1], [0, 1, 0]])

test_act = np.random.random(size=(8, 4, 4))
test_fb = net.mstd_fb(t=1, curr_mstd_out=test_act)
print('Your feedback signal from MSTd (1st 2 directions) is:\n', test_fb[:2])

You should get the following output:

    Your feedback signal from MSTd (1st 2 directions) is:
     [[[12.499 17.705 19.242 15.673]
      [17.464 21.184 22.453 19.604]
      [21.062 22.546 23.542 20.478]
      [18.182 19.925 21.551 18.902]]

     [[12.181 19.097 20.065 15.029]
      [17.794 25.019 24.393 18.994]
      [21.355 24.216 24.465 18.923]
      [16.833 18.432 18.755 16.604]]]

#### (iv) Implement Layer 6 (MSTd)

This now follows the usual workflow:

- Update `make_kernels` to build Level 6 inhibitory feedback kernel. Filter size should be same size as the long-range filter.
- Update `d_long_range` to include this feedback signal if we are simulating Layer 6. **NOTE:** When computing the MSTd feedback signal, it should be based on the output signal at $t-1$. Why? `self.mstd_out[t]` should be undefined (we compute Level 5 before Level 6).
- Implement `d_mstd_grouping` (Level 6 derivatives).
- Update `update_net` to integrate cell activity in Level 6 (remember to put your code in a boolean if statement).
- Simulate the noise-free rightward motion RDK video (`test_rdk_0`) with all network layers.
- Run the direction decode test below.
- In the cell below, rectify the MSTd cells.
- Plot the activation of the rectified output of all directions using `plot_act_image_grid`. Compare for consistency to `right_test_level6_grid.mov`. Note that I am plotting starting at time step 9 to the end in steps of 10. (*Please ignore the drifting of the placement of the second row of plots*).
- Plot the rectified MSTd cells using your vector sum plot. Compare for consistency with `right_test_level6_vectors.mov`

In [ ]:

# Set up input
np.random.seed(0)

# Define parameters

# Simulate net

# Compute rectified layer output


In [ ]:
# Your grid of image plots of the layer 6 cells output over time


#### Test: Decoded directional transient directions

**IMPORTANT NOTE:** Your exact values **WILL** differ and thats ok (due to parameter choices and different implementation decisions). You simply want to get rough qualitative agreement.

In [ ]:
print(f'Your decoded evidence for each direction at the end of the simulation is\n{net.decode_direction(net.mstd_cells, -1)}\nand should roughly look like\n[402.962   0.149   0.      0.047   0.      0.453   0.17    0.027]')

In [ ]:
# Your vector sum plot of the layer 6 output over time


## Task 5) Direction discrimination with noise

- Perform your own full network simulations to discriminate the coherent motion direction in videos with random noise (uncorrelated frame-to-frame) and correlated every 3 frames.
- Show dominant direct decoded direction and make plots of each network stage.

### 5a. Simulations with uncorrelated frame-to-frame noise.

Generate new random dot kinematograms with noise (that you already implemented support for). Test out the full network with noise, visualize the estimated motion, and print out the decoded evidence for different motion directions.

### 5b. Simulations with complex noise: coherence videos.

- Import the 10 videos with different levels of coherence (in `coherence_stimuli.mat`, which is in MATLAB format. Check out `loadmat` function). In these videos, three coherent motion videos are interleaved (e.g. frame 1, 4, 7, ... go together, 2, 5, 8, ... go together, etc.). The interleaving makes it challenging for the network to detect the percieved motion because dots don't correspond to each other frame-to-frame, yet we still percieve motion!
- There are 10 levels of dot coherence (100% coherence = no noise, 0% coherence = all noise). Test out the network like in 5a in SOME of these videos (doesn't need to be all).

## Extensions

### 1. Additional visualizations

Neural network simulations with many areas and dimensions (space, motion direction, etc) present challenges to interpreting and visualizing the dynamics. Extensions like the following ideas are aimed to give us a better picture of the network's operation.

#### Needle plot

- Visualize the activity of ALL direction cells at ALL spatial positions with a needle plot. For each position (x, y), draw line segments (no arrowheads) coming out of (x,y) aligned each of the 8 preferred directions (e.g. for rightward motion, line sticks to the right, for up-and-right motion line sticks +45 deg, etc). The length is proportioonal to the cell activity. Do whatever normalization to make things look good/helpful. Make your plot show activations over time.

#### Mean activity plots

- Plot the mean direction activity across space for each area.

#### Better vector sum plots

- Superimpose the motion vectors onto of the input dot patterns so that you can see the correspondance.

### 2. Additional analyses

Analyze the network in new ways. Many ways to do this, here are some ideas:

- Plot/interpret the motion direction evidence over time in different areas.
- Compute the relative SNR (signal to noise ratio) in different areas for the dominant motion vs rest.

### 3. Additional RDKs

- Analyze/simulate your network with RDKs with different movement directions, changing directions, two simultaneous directions (called transparent motion), etc...lots of possibilities.

### 4. Different motion detection mechanisms

- We used nulling inhibition to detect motion. You should be able to "swap in" other motion detection mechanisms (for example, in easiest-to-hardest to implement order, Reichardt, Gradient, Spatio-temporal energy, etc).
- For Reichardt (correlation-based), the easiest approach that tends to work well is to add the pre and post movement activations together in whatever the neuron's prefered direction is, then apply a threshold to filter out low matches. For example, to detect rightward motion, you could do something like $x_i(t-1) + x_{i+1}(t)$ then compare that to an activation threshold.

### 5. Support for multiple speeds

- (Challenging) This is more open ended. A simplification we made to keep the project scope reasonable is to only support cells that only detect 1 px/frame motion. Experiment, brainstorm, and implement support for other speeds (2 px/frame, 3 px/frame, etc).